In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal
from enum import Enum
from collections import namedtuple

# 外皮性能の計算方法 (関数)

## はじめに

本計算は第3章「暖冷房負荷と外皮性能」第2章「外皮性能」にもとづく。

## 型について

### 地域の区分に関する列挙型

地域の区分8地域を表す。

In [2]:
class Region(Enum):
    Division1 = 1
    Division2 = 2
    Division3 = 3
    Division4 = 4
    Division5 = 5
    Division6 = 6
    Division7 = 7
    Division8 = 8

### orientation 型 (簡易法のみに使用)

主開口方位から時計回りに0°、90°、180°、270°を表す。省エネ基準における簡易評価法においては、主開口方位は南西と固定されているため、順に、南西、北西、北東、南東の意味と同じである。

In [3]:
class orientation:
    def __init__(self, D0, D90, D180, D270):
        self.D0   = D0
        self.D90  = D90
        self.D180 = D180
        self.D270 = D270
    
    def sum(self):
        return self.D0 + self.D90 + self.D180 + self.D270
    
    def array(self):
        return np.array([self.D0, self.D90, self.D180, self.D270])

In [4]:
Orientation = namedtuple('Orientation', 'D0 D90 D180 D270')

### 方位に関する列挙型

基準で扱う方位は8方位と上面・下面である。

In [5]:
class Direction(Enum):
    S  = 0
    SW = 1
    W  = 2
    NW = 3
    N  = 4
    NE = 5
    E  = 6
    SE = 7
    TOP = 8
    Bottom = 9

### 隣接空間に関する列挙型

隣接空間の種類は付録Bで定められている。  
ここでは、以下のようにネーミングする。  
* External: 「外気」「外気に通じる空間」
* SemiExternal: 「外気に通じていない空間」「外気に通じる床裏」
* Internal: 「住戸及び住戸と同様の熱的環境の空間」「外気に通じていない床裏」

In [6]:
class AdjacentSpaceType(Enum):
    External     = 0
    SemiExternal = 1
    Internal     = 2

## 外皮の部位について

外皮の部位は、外壁(界壁)・天井・屋根・床(界床)から成る一般部位と、開口部、土間床で構成される。また、熱の計算においては、外皮の部位を面的に流れる熱流に加え、RC造やS造の隅角部等における熱橋計算および土間床外周部の熱計算を行う必要がある。  
それぞれ、「外皮の部位」「熱橋」「土間床外周部」とする。

### 外皮の部位 (詳細法)

外皮の部位は、面積・熱損失係数(U値)・暖房期の日射熱取得率・冷房期の日射熱取得率・方位・隣接空間の種類をもつ。

In [7]:
EnvPart = namedtuple("EnvPart", "A U eta_H eta_C direction ASType")
# A: 面積(m2)
# U: 熱損失係数(W/m2K)
# eta_H: 暖房期の日射熱取得率(-)
# eta_C: 冷房期の日射熱取得率(-)
# directino: 方位 [direction列挙型]
# ASType: 隣接空間の種類 [AdjacentSpaceType型]

### 外皮の部位 (簡易法)

外皮の部位の簡易法版。詳細法版と異なり、面積・方位をもたない。

In [8]:
SimpleEnvPart = namedtuple("SimpleEnvPart", "U eta_H eta_C ASType")
# U: 熱損失係数(W/m2K)
# eta_H: 暖房期の日射熱取得率(-)
# eta_C: 冷房期の日射熱取得率(-)
# ASType: 隣接空間の種類 [AdjacentSpaceType型]

### 外皮の部位(窓)(簡易法)

外皮の部位(窓)の簡易法版。簡易法に限り、窓の$U$値、$\eta_H$値、$\eta_C$値、方位、隣接空間の種類に加えて、$\eta_d$値、$f_H$値、$f_C$値が必要である。簡易法の場合、最も不利側の$\eta_d$値、$f$値を計算する必要があるため。ただし、$f$値に関しては、デフォルト値を適用する場合、入力する必要はない。

In [9]:
SimpleEnvWnd = namedtuple("SimpleEnvWnd", "U eta_H eta_C ASType eta_d f_H f_C")
# U: 熱損失係数(W/m2K)
# eta_H: 暖房期の日射熱取得率(-)
# eta_C: 冷房期の日射熱取得率(-)
# ASType: 隣接空間の種類 [AdjacentSpaceType型]
# eta_d: 垂直面日射熱取得率(-)
# f_H: 暖房期の取得日射熱補正係数(-)
# f_C: 冷房期の取得日射熱補正係数(-)

### 熱橋 (詳細法)

熱橋は、長さ・線熱損失係数・暖房期の日射熱取得率・冷房期の日射熱取得率・方位1と方位2・隣接空間の種類をもつ。熱橋は多くの場合、隅角部等に発生し、2方位に面する場合が多いため、方位1と方位2を持つことにする。方位1と方位2に順番は無い。また、T型熱橋など、接続する部位が同じ方位を持つ場合は、方位1と方位2に同じ方位を入力すること。  
ただし、熱橋の日射熱取得率は、本来であれば、  
$\begin{align} \eta = \gamma \times 0.034 \times \frac{ L \times \psi }{ A } \end{align}$  
で表されるのに対し、  
$\begin{align} \eta' = \gamma \times 0.034 \times \psi \end{align} $  
で表されることとする。  
ここで、  
$\eta$: 日射熱取得率(-)  
$\eta'$: 熱橋の日射熱取得率(m)  
従って、$ m $ 値(m<sup>2</sup>)の計算において、仕様書によると、  
$\begin{align} m = A \times \eta \times \nu \end{align} $  
ではなく、  
$\begin{align} m = L \times \eta' \times \nu \end{align} $  
で計算される。

In [10]:
EnvHB = namedtuple("EnvHB", "L psi eta_H_dash eta_C_dash direction1 direction2 ASType")
# L: 長さ(m)
# psi: 線熱損失係数(W/mK)
# eta_H: 暖房期の熱橋の日射熱取得率(m)
# eta_C: 冷房期の熱橋の日射熱取得率(m)
# direction1: 方位1 [direction列挙型]
# direction2: 方位2 [direction列挙型]
# ASType: 隣接空間の種類 [AdjacentSpaceType型]

### 熱橋 (簡易法)

熱橋の簡易法版はない。現時点で、簡易法は木造戸建て住宅のみ認められている。木造戸建て住宅は熱橋計算が不要なため、簡易法では熱橋を持つ必要はない。

### 土間床外周部 (詳細法)

土間床外周部は、長さ・線熱損失係数・隣接空間の種類をもつ。日射に関する熱の移動はない。従って、方位の概念も持たない。

In [11]:
EnvEFPeri = namedtuple("EnvEFPeri", "L psi ASType")
# L: 長さ(m)
# psi: 線熱損失係数(W/mK)
# ASType: 隣接空間の種類 [AdjacentSpaceType型]

### 土間床外周部 (簡易法)

土間床外周部の簡易法版。詳細法版と違い、長さをもたない。

In [12]:
SimpleEnvEFPeri = namedtuple("SimpleEnvEFPeri", "psi ASType")
# psi: 線熱損失係数(W/mK)
# ASType: 隣接空間の種類 [AdjacentSpaceType型]

## 熱損失係数(換気による熱損失を含まない)

#### 定義

熱損失係数(換気による熱損失を含まない)$Q'$(W/m<sup>2</sup>K)は式(1)に基づく。

In [13]:
def get_Q_dash(U_A, r_env):
    # U_A: 外皮平均熱損失係数(W/m2K)
    # r_env: 床面積の合計に対する外皮の部位の面積の合計の比
    return U_A * r_env

#### 説明

外皮平均熱貫流率は、単位温度差あたりの熱損失量 $q$ を床面積の合計で除した値である。  
$ U_A = q \div A_f $  
熱損失係数は、単位温度差あたりの熱損失量 $q$ を外皮の部位の面積の合計で除した値である。  
$ Q' = q \div A_{env} $  
床面積の合計に対する外皮の部位の面積の合計の比 $r_{env}$ は、 $U_A$ 値を $Q'$ に変換する係数である。

In [14]:
get_Q_dash( 0.87 , 307.51/120.08 )

2.227962191872085

## 日射取得係数

#### 定義

暖房期の日射取得係数($\mu_H$)は式(2)に基づく。

In [15]:
def getMuH(eta_A_H, r_env):
    # eta_A_H: 暖房期の日射取得係数(-)
    # r_env: 床面積の合計に対する外皮の部位の面積の合計の比
    return eta_A_H * r_env / 100

冷房期の日射取得係数($\mu_C$)は式(3)に基づく。

In [16]:
def getMuC(eta_A_C, r_env):
    # eta_A_C: 冷房期の日射取得係数(-)
    # r_env: 床面積の合計に対する外皮の部位の面積の合計の比
    return eta_A_C * r_env / 100

#### 説明

In [17]:
getMuH(4.3, 307.51/120.08 )

0.11011767155229846

In [18]:
getMuC(2.8, 307.51/120.08 )

0.07170453031312457

## 当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法

### 外皮平均熱貫流率

#### 定義

外皮平均熱貫流率($U_A$)は式(4)に基づく。得られた値の100分の1未満の端数は切り上げ、小数第2位までの値とする。

In [19]:
def get_UA( U_parts, psi_parts, A_env ):
    # U_parts: 外皮の部位 [ U_part の配列 ]
    #          U_part: 外皮の部位 ( 面積(m2), 熱貫流率(W/m2K), 温度差係数(-) )
    # psi_parts: 熱橋等 [ psi_part の配列 ]
    #            psi_part: 熱橋等 ( 長さ(m), 線熱貫流率(W/mK), 温度差係数(-) )
    # A_env: 外皮の部位の面積の合計(m2)
    return Decimal( (   sum([U_part[0]   * U_part[1]   * U_part[2]   for U_part   in U_parts  ])
                      + sum([psi_part[0] * psi_part[1] * psi_part[2] for psi_part in psi_parts])
                    ) / A_env
                  ).quantize( Decimal('0.00'), rounding = 'ROUND_UP' )

#### 説明

例題として、以下の住宅で計算する。  
* 4m × 4m × 2.5m の基礎断熱住宅（屋根 16 m<sup>2</sup>、 壁 10 m<sup>2</sup> が4枚、 地盤の周囲は 4m が4か所）
* $U$値は1.0
* $\psi$値は1.8
* 温度差係数は1.0  
外皮の面積の合計には、熱損失はゼロである土間床も含まれる。

In [20]:
# 配列の要素は、Uに関しては(壁,壁,壁,壁,天井)、Lに関しては(地盤周囲 × 4)
get_UA( [(10.0, 1.0, 1.0), (10.0, 1.0, 1.0), (10.0, 1.0, 1.0), (10.0, 1.0, 1.0), (16.0, 1.0, 1.0)],
     [(4.0, 1.8, 1.0), (4.0, 1.8, 1.0), (4.0, 1.8, 1.0)],
     72.0 )

Decimal('1.08')

### 暖房期の平均日射熱取得率及び冷房期の平均日射熱取得率

#### 定義

暖房期の平均日射熱取得率($\eta_{A,H}$)は式(5)に基づく。得られた値の10分の1未満の端数は切り下げ、小数第1位までの値とする。

In [21]:
def get_eta_A_H(parts, partHBs, A_env):
    # parts: 外皮の部位 [part の配列]
    #        part: 外皮の部位 ( 面積(m2), 暖房期の日射熱取得率(-), 暖房期の方位係数(-) )
    # partHBs: 熱橋 [partHB の配列]
    #          partHB: 熱橋 ( 長さ(m), 暖房期の熱橋の日射熱取得率(m), 暖房期の方位係数1(-), 暖房期の方位係数2(-) )
    # A_env: 外皮の部位の面積の合計(m2)
    return Decimal( (   sum([ part[0]   * part[1]   * part[2]                     for part   in parts   ])
                      + sum([ partHB[0] * partHB[1] * ( partHB[2] + partHB[3] )/2 for partHB in partHBs ])
                    ) / A_env * 100
                   ).quantize(Decimal('0.0'), rounding = 'ROUND_DOWN')

冷房期の平均日射熱取得率($\eta_{C,H}$)は式(6)に基づく。得られた値の10分の1未満の端数は切り上げ、小数第1位までの値とする。

In [22]:
def get_eta_A_C(parts, partHBs, A_env):
    # parts: 外皮の部位 [part の配列]
    #        part: 外皮の部位 ( 面積(m2), 冷房期の日射熱取得率(-), 冷房期の方位係数(-) )
    # partHBs: 熱橋 [partHB の配列]
    #          partHB: 熱橋 ( 長さ(m), 冷房期の熱橋の日射熱取得率(m), 冷房期の方位係数1(-), 冷房期の方位係数2(-) )
    # A_env: 外皮の部位の面積の合計(m2)
    return Decimal( (   sum([ part[0]   * part[1]   * part[2]                     for part   in parts   ])
                      + sum([ partHB[0] * partHB[1] * ( partHB[2] + partHB[3] )/2 for partHB in partHBs ])
                    ) / A_env * 100 
                  ).quantize(Decimal('0.0'), rounding = 'ROUND_UP')

#### 説明

例題として、以下の住宅で計算する。  
* 4m × 4m × 2.5m の基礎断熱住宅（屋根 16 m<sup>2</sup>、 壁 10 m<sup>2</sup> が4枚）
* $\eta$値は暖房期・冷房期ともに0.034  
* $\eta'$値は暖房期・冷房期ともに0.05
* 方位係数は6地域の東西南北とする。つまり、暖房期は、東(0.579)、西(0.523)、南(0.936)、北(0.261)、冷房期は、東(0.512)、西(0.504)、南(0.434)、北(0.341)、上面は暖房期・冷房期ともに1.0

In [23]:
# 配列の要素は、(壁,壁,壁,壁,天井)
get_eta_A_H( [ (10.0, 0.034, 0.579), (10.0, 0.034, 0.523), (10.0, 0.034, 0.936), (10.0, 0.034, 0.261),
               (16.0, 0.034, 1.0) ], 
             [ (2.5, 0.05, 0.579, 0.523), (2.5, 0.05, 0.523, 0.936), (2.5, 0.05, 0.936, 0.261), (2.5, 0.05, 0.261, 0.579),
               (2.0, 0.05, 0.579, 1.0), (2.0, 0.05, 0.523, 1.0), (2.0, 0.05, 0.936, 1.0), (2.0, 0.05, 0.261, 1.0) ],
             72.0 )

Decimal('2.6')

In [24]:
# 配列の要素は、(壁,壁,壁,壁,天井)
get_eta_A_C( [ (10.0, 0.034, 0.512), (10.0, 0.034, 0.504), (10.0, 0.034, 0.434), (10.0, 0.034, 0.341),
               (16.0, 0.034, 1.0) ],
             [ (2.5, 0.05, 0.512, 0.504), (2.5, 0.05, 0.504, 0.434), (2.5, 0.05, 0.434, 0.341), (2.5, 0.05, 0.341, 0.512),
               (2.0, 0.05, 0.512, 1.0), (2.0, 0.05, 0.523, 1.0), (2.0, 0.05, 0.936, 1.0), (2.0, 0.05, 0.261, 1.0) ],
             72.0 )

Decimal('2.4')

### 床面積の合計に対する外皮の部位の面積の合計の比

#### 定義

床面積の合計に対する外皮部位比($r_{env}$)は式(7)に基づく。

In [25]:
def get_r_env(A_env, A_A):
    # A_env: 外皮の部位の面積の合計(m2)
    # A_A: 当該住戸の床面積の合計(m2)
    return A_env / A_A

#### 説明

In [26]:
get_r_env( 307.51, 120.08 )

2.560876082611592

### 外皮の部位の面積の合計

#### 定義

外皮の部位の面積の合計($A_{env}$)は式(8)に基づく。

In [27]:
def get_A_env(A, A_EF):
    # A: 外皮の部位(一般部位又は開口部)の面積 [配列]
    # A_EF: 土間床等の面積 [配列]
    return sum(A) + sum(A_EF)

#### 説明

例題として、以下の住宅で計算する。  
* 4m × 4m × 2.5m の基礎断熱住宅（屋根 16 m<sup>2</sup>、 壁 10 m<sup>2</sup> が4枚、 地盤 16 m<sup>2</sup>）

In [28]:
get_A_env( [10.0, 10.0, 10.0, 10.0, 16.0], [16.0] )

72.0

## 当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法

### 外皮平均熱貫流率(簡易)

#### 定義

外皮平均熱貫流率$U_A$は、式(9a)及び式(9b)に基づく。得られた値の100分の1未満の端数は切り上げ、小数第2位までの値とする。

In [29]:
def simple_U_A (A_roof, A_wall, A_door, A_wnd, A_IF, A_base, A_base_IS, A_base_d, A_base_d_IS, L_prm, L_prm_IS, L_prm_d, L_prm_d_IS, H_roof, H_wall, H_door, H_wnd, H_floor, H_base_OS, H_base_IS, H_prm_OS, H_prm_IS, U_roof, U_wall, U_door, U_wnd, U_floor, U_base, U_base_d, psi_prm, psi_prm_d, A_env ):
    # A_roof: 標準住戸における屋根又は天井の面積 (m2)
    # A_wall: 標準住戸における壁の面積 (m2) [orientation型]
    # A_door: 標準住戸におけるドアの面積 (m2) [orientation型]
    # A_wnd: 標準住戸における窓の面積 (m2) [orientation型]
    # A_IF: 標準住戸における床断熱した床の面積 (m2)
    # A_base: 標準住戸における玄関等を除く基礎の面積 (m2) [orientation型]
    # A_base_IS: 標準住戸における床下に面した玄関等を除く基礎の面積 (m2)
    # A_base_d: 標準住戸における玄関等の基礎の面積 (m2) [orientation型]
    # A_base_d_IS: 標準住戸における床下に面した玄関等の基礎の面積 (m2)
    # L_prm: 標準住戸における玄関等を除く土間床等の外周部の長さ (m) [orientation型]
    # L_prm_IS: 標準住戸における床下に面した玄関等を除く土間床等の外周部の長さ (m)
    # L_prm_d: 標準住戸における玄関等の土間床等の外周部の長さ (m) [orientation型]
    # L_prm_d_IS: 標準住戸における床下に面した玄関等の土間床等の外周部の長さ (m)
    # H_roof: 屋根又は天井の温度差係数
    # H_wall: 壁の温度差係数
    # H_door: ドアの温度差係数
    # H_wnd: 窓の温度差係数
    # H_floor: 床の温度差係数
    # H_base_OS: 外気に面した基礎の温度差係数
    # H_base_IS: 床下に面した基礎の温度差係数
    # H_prm_OS: 外気に面した土間床等の周辺部の温度差係数
    # H_prm_IS: 床下に面した土間床等の周辺部の温度差係数
    # U_roof: 屋根又は天井の熱貫流率(W/m2K)
    # U_wall: 壁の熱貫流率(W/m2K)
    # U_door: ドアの熱貫流率(W/m2K)
    # U_wnd: 窓の熱貫流率(W/m2K)
    # U_floor: 床の熱貫流率(W/m2K)
    # U_base: 玄関等を除く基礎の熱貫流率(W/m2K)
    # U_base_d: 玄関等の基礎の熱貫流率(W/m2K)
    # psi_prm: 玄関等を除く土間床等の外周部の熱貫流率(W/mK)
    # psi_prm_d: 玄関等を除く土間床等の熱貫流率(W/mK)
    # A_env: 標準住戸における外皮の部位の面積の合計(m2)

    q = A_roof * H_roof * U_roof \
      + A_wall.sum() * H_wall * U_wall \
      + A_door.sum() * H_door * U_door \
      + A_wnd.sum() * H_wnd * U_wnd \
      + A_IF * H_floor * U_floor \
      + ( A_base.sum() * H_base_OS + A_base_IS * H_base_IS ) * U_base \
      + ( A_base_d.sum() * H_base_OS + A_base_d_IS * H_base_IS ) * U_base_d \
      + ( L_prm.sum() * H_prm_OS + L_prm_IS * H_prm_IS ) * psi_prm \
      + ( L_prm_d.sum() * H_prm_OS + L_prm_d_IS * H_prm_IS ) * psi_prm_d        
    
    return Decimal(q / A_env).quantize(Decimal('0.00'), rounding='ROUND_UP')

#### 説明

面積・温度差係数は、後述するように、床断熱住宅と基礎断熱住宅でそれぞれデフォルト値が定義されている。例として、ここでは床断熱住宅の値を用いる。  
部位の熱貫流率・線熱貫流率の値は、全て1.0とする。

In [30]:
simple_U_A (50.85, orientation(30.47, 22.37, 47.92, 22.28), orientation(0.0, 1.89, 1.62, 0.0), orientation(22.69, 2.38, 3.63, 4.37), 45.05, orientation(0.00, 0.91, 0.91, 0.00), 1.82, orientation(0.00, 0.33, 0.25, 0.00), 0.57, orientation(0.00, 1.82, 1.82, 0.00), 3.64, orientation(0.00, 1.82, 1.37, 0.00), 3.19, 1.0, 1.0, 1.0, 1.0, 0.7, 1.0, 0.7, 1.0, 0.7, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 266.10 )

Decimal('0.97')

### 暖房期の平均日射熱取得率及び冷房期の平均日射熱取得率(簡易)

#### 定義

暖房期の平均日射熱取得率($\eta_{A,H}$)は式(10a)及び式(10b)に基づく。得られた値の10分の1未満の端数は切り下げ、小数第1位までの値とする。

In [31]:
def simple_eta_A_H(A_roof, A_wall, A_door, A_wnd, A_base, A_base_d, nu_H_top, nu_H, eta_H_roof, eta_H_wall, eta_H_door, eta_H_wnd, eta_H_base, eta_H_base_d, A_env):
    # A_roof: 標準住戸における屋根又は天井の面積(m2)
    # A_wall: 標準住戸における壁の面積(m2) [orientation型]
    # A_door: 標準住戸におけるドアの面積(m2) [orientation型]
    # A_wnd: 標準住戸における窓の面積(m2) [orientation型]
    # A_base: 標準住戸における玄関等を除く基礎の面積(m2) [orientation型]
    # A_base_d: 標準住戸における玄関の基礎の面積(m2) [orientation型]
    # nu_H_top; 上面に面した外皮の部位の暖房期の方位係数
    # nu_H: 外皮の部位の暖房期の方位係数 [orientation型]
    # eta_H_roof: 屋根又は天井の暖房期の日射熱取得率
    # eta_H_wall: 壁の暖房期の日射熱取得率
    # eta_H_door: ドアの暖房期の日射熱取得率
    # eta_H_wnd: 窓の暖房期の日射熱取得率 [orientation型]
    # eta_H_base: 玄関等を除く基礎の暖房期の日射熱取得率
    # eta_H_base_d: 玄関等の基礎の暖房期の日射熱取得率
    # A_env: 標準住戸における外皮の部位の面積の合計(m2)
    m_H = A_roof * nu_H_top * eta_H_roof \
        + ( A_wall.array() * nu_H.array() ).sum() * eta_H_wall \
        + ( A_door.array() * nu_H.array() ).sum() * eta_H_door \
        + ( A_wnd.array() * nu_H.array() * eta_H_wnd.array() ).sum() \
        + ( A_base.array() * nu_H.array() ).sum() * eta_H_base \
        + ( A_base_d.array() * nu_H.array() ).sum() * eta_H_base_d

    return Decimal( m_H / A_env * 100 ).quantize(Decimal('0.0'), rounding = 'ROUND_DOWN')

In [32]:
def simple_eta_A_C(A_roof, A_wall, A_door, A_wnd, A_base, A_base_d, nu_C_top, nu_C, eta_C_roof, eta_C_wall, eta_C_door, eta_C_wnd, eta_C_base, eta_C_base_d, A_env):
    # A_roof: 標準住戸における屋根又は天井の面積(m2)
    # A_wall: 標準住戸における壁の面積(m2) [orientation型]
    # A_door: 標準住戸におけるドアの面積(m2) [orientation型]
    # A_wnd: 標準住戸における窓の面積(m2) [orientation型]
    # A_base: 標準住戸における玄関等を除く基礎の面積(m2) [orientation型]
    # A_base_d: 標準住戸における玄関の基礎の面積(m2) [orientation型]
    # nu_C_top; 上面に面した外皮の部位の冷房期の方位係数
    # nu_C: 外皮の部位の冷房期の方位係数 [orientation型]
    # eta_C_roof: 屋根又は天井の冷房期の日射熱取得率
    # eta_C_wall: 壁の冷房期の日射熱取得率
    # eta_C_door: ドアの冷房期の日射熱取得率
    # eta_C_wnd: 窓の冷房期の日射熱取得率 [orientation型]
    # eta_C_base: 玄関等を除く基礎の冷房期の日射熱取得率
    # eta_C_base_d: 玄関等の基礎の冷房期の日射熱取得率
    # A_env: 標準住戸における外皮の部位の面積の合計(m2)
    m_C = A_roof * nu_C_top * eta_C_roof \
        + ( A_wall.array() * nu_C.array() ).sum() * eta_C_wall \
        + ( A_door.array() * nu_C.array() ).sum() * eta_C_door \
        + ( A_wnd.array() * nu_C.array() * eta_C_wnd.array() ).sum() \
        + ( A_base.array() * nu_C.array() ).sum() * eta_C_base \
        + ( A_base_d.array() * nu_C.array() ).sum() * eta_C_base_d

    return Decimal( m_C / A_env * 100 ).quantize(Decimal('0.0'), rounding = 'ROUND_UP')

#### 説明

面積は、後述するように、床断熱住宅と基礎断熱住宅でそれぞれデフォルト値が定義されている。例として、ここでは床断熱住宅の値を用いる。  
* 非透明部位の日射熱取得率は0.34とし、透明部位の日射熱取得率は0.5とする。  
* 線熱貫流率の値は、全て1.0とする。
* 方位係数は6地域の値とし、方位は、順に、南西・北西・北東・南東の方位係数を使用した。

In [33]:
simple_eta_A_H(50.85, orientation(30.47, 22.37, 47.92, 22.28), orientation(0.0, 1.89, 1.62, 0.0), orientation(22.69, 2.38, 3.63, 4.37), orientation(0.00, 0.91, 0.91, 0.00), orientation(0.00, 0.33, 0.25, 0.00), 1.0, orientation(0.763, 0.317, 0.325, 0.833), 0.034, 0.034, 0.034, orientation(0.5, 0.5, 0.5, 0.5), 0.034, 0.034, 266.10)

Decimal('5.7')

In [34]:
simple_eta_A_C(50.85, orientation(30.47, 22.37, 47.92, 22.28), orientation(0.0, 1.89, 1.62, 0.0), orientation(22.69, 2.38, 3.63, 4.37), orientation(0.00, 0.91, 0.91, 0.00), orientation(0.00, 0.33, 0.25, 0.00), 1.0, orientation(0.491, 0.427, 0.431, 0.498), 0.034, 0.034, 0.034, orientation(0.5, 0.5, 0.5, 0.5), 0.034, 0.034, 266.10)

Decimal('4.4')

### 床面積の合計に対する外皮の部位の面積の合計の比

#### 定義

床面積の合計に対する外皮の部位の面積の合計の比 $r_{env}$は式(12)に基づく。この式は、式(7)と同じである。

### 標準住戸の種類

#### 定義

標準住戸の面積は、住宅の種類（「床断熱住戸」又は「基礎断熱住戸」）に依存する。これらに、床断熱住戸と基礎断熱住戸の両方の条件を満たす住戸（「床断熱と基礎断熱の併用住戸」という。）を加えて、住宅の種類を定義する。

In [35]:
class simple_HouseType(Enum):
    floor_ins          = 0
    base_ins           = 1
    floor_and_base_ins = 2

### 標準住戸における外皮の部位の面積及び土間床等の外周部の長さ等

#### 定義

標準住戸における外皮の部位の面積及び土間床等の外周部の長さ等は、表3に基づく。

In [36]:
class simple_Area_and_Length:
    
    def __init__(self, house_type):
        # house_type: HouseType 列挙型

        if house_type == simple_HouseType.floor_and_base_ins:
            rase("Not Defined")

        # 外皮の部位の面積の合計(m2)
        self.env    = [ 266.10, 275.69 ][house_type.value]

        # 延べ床面積の合計(m2)
        self.A      = 90.0
        
        # 屋根又は天井の面積(m2)
        self.roof   = 50.85
        
        # 壁の面積(m2)
        self.wall   = orientation( 30.47, 22.37, 47.92, 22.28 )

        # ドアの面積(m2)
        self.door   = orientation( 0.00, 1.89, 1.62, 0.00 )

        # 窓の面積(m2) 
        self.wnd    = orientation( 22.69, 2.38, 3.63, 4.37 )
        
        # 床（床断熱した床）の面積(m2)
        self.floor  = [ 45.05, 0.0 ][house_type.value]

        # 玄関等を除く基礎の面積(m2)
        self.base   = [ orientation(  0.00, 0.91, 0.91, 0.00 ),
                        orientation(  5.30, 1.48, 4.62, 2.40 ) ][house_type.value]
        self.base_IS = [ 1.82, 0.00 ][house_type.value]

        # 玄関等の基礎の面積(m2)
        self.base_d = [ orientation(  0.00, 0.33, 0.25, 0.00 ),
                        orientation(  0.00, 0.33, 0.25, 0.00 )][house_type.value]
        self.base_d_IS = [ 0.57, 0.00 ][house_type.value]
        
        # 玄関等を除く土間床等の外周部の長さ(m)        
        self.prm = [ orientation(  0.00, 1.82, 1.82, 0.00 ),
                     orientation( 10.61, 2.97, 9.24, 4.79 ) ][house_type.value]
        self.prm_IS = [ 3.64, 0.00 ][house_type.value]
        
        # 玄関等の土間床等の外周部の長さ(m)
        self.prm_d  = [ orientation(  0.00, 1.82, 1.37, 0.00 ),
                        orientation(  0.00, 1.82, 1.37, 0.00 ) ][house_type.value]
        self.prm_d_IS = [ 3.19, 0.00 ][house_type.value]        

#### 説明

値の確認。

床断熱住戸の場合

In [37]:
al_f = simple_Area_and_Length(simple_HouseType.floor_ins)
print(al_f.env)
print(al_f.A)
print(al_f.roof)
print(al_f.wall.array())
print(al_f.door.array())
print(al_f.wnd.array())
print(al_f.floor)
print(al_f.base.array())
print(al_f.base_IS)
print(al_f.base_d.array())
print(al_f.base_d_IS)
print(al_f.prm.array())
print(al_f.prm_IS)
print(al_f.prm_d.array())
print(al_f.prm_d_IS)

266.1
90.0
50.85
[ 30.47  22.37  47.92  22.28]
[ 0.    1.89  1.62  0.  ]
[ 22.69   2.38   3.63   4.37]
45.05
[ 0.    0.91  0.91  0.  ]
1.82
[ 0.    0.33  0.25  0.  ]
0.57
[ 0.    1.82  1.82  0.  ]
3.64
[ 0.    1.82  1.37  0.  ]
3.19


基礎断熱住戸の場合

In [38]:
al_b = simple_Area_and_Length(simple_HouseType.base_ins)
print(al_b.env)
print(al_b.A)
print(al_b.roof)
print(al_b.wall.array())
print(al_b.door.array())
print(al_b.wnd.array())
print(al_b.floor)
print(al_b.base.array())
print(al_b.base_IS)
print(al_b.base_d.array())
print(al_b.base_d_IS)
print(al_b.prm.array())
print(al_b.prm_IS)
print(al_b.prm_d.array())
print(al_b.prm_d_IS)

275.69
90.0
50.85
[ 30.47  22.37  47.92  22.28]
[ 0.    1.89  1.62  0.  ]
[ 22.69   2.38   3.63   4.37]
0.0
[ 5.3   1.48  4.62  2.4 ]
0.0
[ 0.    0.33  0.25  0.  ]
0.0
[ 10.61   2.97   9.24   4.79]
0.0
[ 0.    1.82  1.37  0.  ]
0.0


### 床断熱住宅か基礎断熱住宅の判断

#### 定義

「9.5 標準住戸における外皮の部位の面積及び土間床等の外周部の長さ等」において、「床断熱住戸」及び「基礎断熱住戸」のどちらにも該当する場合、両方の場合で外皮平均熱貫流率を計算し、値が大きい方の場合を採用することとなっている。

In [39]:
def simple_house_type_judge(floor_ins_U_A, base_ins_U_A):
    # floor_ins_U_A: 床断熱住戸のUA値(W/m2K)
    # base_ins_U_A: 基礎熱住戸のUA値(W/m2K)
    return simple_HouseType.floor_ins if floor_ins_U_A > base_ins_U_A else simple_HouseType.base_ins

#### 説明

床断熱住戸の方のUA値が大きい場合。

In [40]:
simple_house_type_judge(2.2, 1.1)

<simple_HouseType.floor_ins: 0>

基礎熱住戸の方のUA値が大きい場合。

In [41]:
simple_house_type_judge(1.1, 2.2)

<simple_HouseType.base_ins: 1>

### 外皮の部位及び土間床等の周辺部の温度差係数

#### 定義

外皮の部位及び土間床等の周辺部の温度差係数は表4に基づく。

In [42]:
class simple_HValue:
    roof    = 1.0
    wall    = 1.0
    door    = 1.0
    wnd     = 1.0
    floor   = 0.7
    base_OS = 1.0
    base_IS = 0.7
    prm_OS  = 1.0
    prm_IS  = 0.7

#### 説明

確認

In [43]:
simple_HValue.roof

1.0

In [44]:
simple_HValue.wall

1.0

In [45]:
simple_HValue.door

1.0

In [46]:
simple_HValue.wnd

1.0

In [47]:
simple_HValue.floor

0.7

In [48]:
simple_HValue.base_OS

1.0

In [49]:
simple_HValue.base_IS

0.7

In [50]:
simple_HValue.prm_OS

1.0

In [51]:
simple_HValue.prm_IS

0.7

### 外皮の部位の熱貫流率

#### 定義

屋根又は天井の熱貫流率($U_{roof}$)は、「9.8.1 屋根又は天井の熱貫流率」に基づく。  
壁の熱貫流率($U_{wall}$)は、「9.8.2 壁の熱貫流率」に基づく。  
ドアの熱貫流率($U_{door}$)は、「9.8.3 ドアの熱貫流率」に基づく。  
窓の熱貫流率($U_{wnd}$)は、「9.8.4 窓の熱貫流率」に基づく。  
床の熱貫流率($U_{floor}$)は、「9.8.5 床の熱貫流率」に基づく。  
玄関等を除く基礎の熱貫流率($U_{base}$)は、「9.8.6 玄関等を除く基礎の熱貫流率」に基づく。  
玄関等の基礎の熱貫流率($U_{base,d}$)は、「9.8.7 玄関等の基礎の熱貫流率」に基づく。  
なお、窓の熱貫流率の2%ルール(床面積の2%に満たない面積の部位は除外できる。)や浴室下部の除外ルール(断熱措置がある場合)等のルールは実装しない。

In [52]:
def simple_U_part(parts):
    # parts: 外皮の部位 [ part の配列 ]
    #        part: 外皮の部位 ( U値(W/m2K), H値 )
    return max ( parts, key = ( lambda part: part[0] * part[1] ) )

simple_U_roof   = simple_U_part
simple_U_wall   = simple_U_part
simple_U_door   = simple_U_part
simple_U_wnd    = simple_U_part
simple_U_floor  = simple_U_part
simple_U_base   = simple_U_part
simple_U_base_d = simple_U_part

#### 説明

計算例。U値が大きい方が選択される例。

In [53]:
print( simple_U_part([ (1.5,1.0), (2.0,1.0)] ) )

(2.0, 1.0)


計算例。H値の違いによりU値が小さい方が選択される例。

In [54]:
simple_U_part([ (1.5,1.0), (2.0,0.7)] )

(1.5, 1.0)

### 土間床等の周辺部の熱貫流率

#### 定義

玄関等を除く土間床等の外周部の線熱貫流率($\psi_{prm}$)は、「9.2.8 玄関等を除く土間床等の外周部の線熱貫流率」に基づく。  
玄関等の土間床等の外周部の線熱貫流率($\psi_{prm,d}$)は、「9.2.9 玄関等の土間床等の外周部の線熱貫流率」に基づく。  

In [55]:
def simple_psi_part(parts):
    # parts: 土間床等の外周部 [ part の配列 ]
    #        part: 土間床等の外周部 ( psi値(W/mK), H値)
    return max ( parts, key = ( lambda part: part[0] * part[1] ) )

simple_psi_prm   = simple_psi_part
simple_psi_prm_d = simple_psi_part

#### 説明

計算例。U値が大きい方が選択される例。

In [56]:
simple_psi_part( [ (1.5,1.0), (2.0,1.0) ] )

(2.0, 1.0)

計算例。H値の違いによりU値が小さい方が選択される例。

In [57]:
simple_psi_part( [ (1.5,1.0), (2.0,0.7) ] )

(1.5, 1.0)

### 外皮の部位(窓を除く)の日射熱取得率

#### 定義

屋根又は天井の日射熱取得率($\eta_{H,roof}$・$\eta_{C,roof}$)は、屋根又は天井の熱貫流率$U_{roof}$を用いて、「9.9.1 屋根又は天井の日射熱取得率」に基づく方法で計算される。  
壁の日射熱取得率($\eta_{H,wall}$・$\eta_{C,wall}$)は、壁の熱貫流率$U_{wall}$を用いて、「9.9.2 壁の日射熱取得率」に基づく方法で計算される。  
ドアの日射熱取得率($\eta_{H,door}$・$\eta_{C,door}$)は、ドアの熱貫流率$U_{door}$を用いて、「9.9.3 ドアの日射熱取得率」に基づく方法で計算される。  
玄関等を除く基礎の日射熱取得率($\eta_{H,base}$・$\eta_{C,base}$)は、玄関等を除く基礎の熱貫流率$U_{base}$を用いて、「9.9.5 玄関等を除く基礎の日射熱取得率」に基づく方法で計算される。  
玄関等の基礎の日射熱取得率($\eta_{H,base,d}$・$\eta_{C,base,d}$)は、玄関等の基礎の熱貫流率$U_{base,d}$を用いて、「9.9.6 玄関等の基礎の日射熱取得率」に基づく方法で計算される。  

In [58]:
def simple_eta_H_not_wnd(U_part):
    # U_part: 外皮の部位(窓を除く)の熱貫流率(W/m2K)
    gamma_H = 1.0 # 暖房期の日除けの効果係数
    return 0.034 * U_part * gamma_H

simple_eta_H_roof   = simple_eta_H_not_wnd
simple_eta_H_wall   = simple_eta_H_not_wnd
simple_eta_H_door   = simple_eta_H_not_wnd
simple_eta_H_base   = simple_eta_H_not_wnd
simple_eta_H_base_d = simple_eta_H_not_wnd

In [59]:
def simple_eta_C_not_wnd(U_part):
    # U_part: 外皮の部位(窓を除く)の熱貫流率(W/m2K)
    gamma_C = 1.0 # 冷房期の日除けの効果係数
    return 0.034 * U_part * gamma_C

simple_eta_C_roof   = simple_eta_C_not_wnd
simple_eta_C_wall   = simple_eta_C_not_wnd
simple_eta_C_door   = simple_eta_C_not_wnd
simple_eta_C_base   = simple_eta_C_not_wnd
simple_eta_C_base_d = simple_eta_C_not_wnd

#### 説明

計算例。U値=1.0の場合。

In [60]:
simple_eta_H_not_wnd(1.0)

0.034

In [61]:
simple_eta_C_not_wnd(1.0)

0.034

### 窓の日射熱取得率

#### 定義

窓の日射熱取得率($\eta_{H,wnd,0}$・$\eta_{H,wnd,90}$・$\eta_{H,wnd,180}$・$\eta_{H,wnd,270}$・$\eta_{C,wnd,0}$・$\eta_{C,wnd,90}$・$\eta_{C,wnd,180}$・$\eta_{C,wnd,270}$)は、「9.9.4 窓の日射熱取得率」に基づく方法で計算される。

In [62]:
def simple_eta_H_wnd(wnds, is_f_value_default, default_f_H_value):
    # wnds: 窓 [wnd の配列]
    #       wnd: 窓 (eta_d値(-), デフォルトのf_H値(-))
    #            デフォルトのf_H値は、is_f_value_default が False の場合は使用しない。
    eta_d_rep = min( [wnd[0] for wnd in wnds ] )
    if( is_f_value_default == True ):
        return Orientation(eta_d_rep * default_f_H_value.D0,
                           eta_d_rep * default_f_H_value.D90,
                           eta_d_rep * default_f_H_value.D180,
                           eta_d_rep * default_f_H_value.D270)
    else:
        f_H_rep = np.min( [wnd[1] for wnd in wnds ] )
        return Orientation(eta_d_rep * f_H_rep,
                           eta_d_rep * f_H_rep,
                           eta_d_rep * f_H_rep,
                           eta_d_rep * f_H_rep)

In [63]:
def simple_eta_C_wnd(wnds, is_f_value_default, default_f_C_value):
    # wnds: 窓 [wnd の配列]
    #       wnd: 窓 (eta_d値(-), デフォルトのf_C値(-))
    #            デフォルトのf_C値は、is_f_value_default が False の場合は使用しない。
    eta_d_rep = max( [wnd[0] for wnd in wnds ] )
    if( is_f_value_default == True ):
        return Orientation(eta_d_rep * default_f_C_value.D0,
                           eta_d_rep * default_f_C_value.D90,
                           eta_d_rep * default_f_C_value.D180,
                           eta_d_rep * default_f_C_value.D270)
    else:
        f_C_rep = np.max( [wnd[1] for wnd in wnds ] )
        return Orientation(eta_d_rep * f_C_rep,
                           eta_d_rep * f_C_rep,
                           eta_d_rep * f_C_rep,
                           eta_d_rep * f_C_rep)

#### 説明

f値にデフォルト値を使用する場合。

In [64]:
simple_eta_H_wnd([(0.5,0.7),(0.6,1.0)], True, Orientation(0.4, 0.3, 0.4, 0.3) )

Orientation(D0=0.2, D90=0.15, D180=0.2, D270=0.15)

In [65]:
simple_eta_C_wnd([(0.5,0.7),(0.6,1.0)], True, Orientation(0.4, 0.3, 0.4, 0.3) )

Orientation(D0=0.24, D90=0.18, D180=0.24, D270=0.18)

f値にデフォルト値を使用しない場合。

In [66]:
simple_eta_H_wnd([(0.5,0.7),(0.6,1.0)], False, Orientation(0.4, 0.3, 0.4, 0.3) )

Orientation(D0=0.34999999999999998, D90=0.34999999999999998, D180=0.34999999999999998, D270=0.34999999999999998)

In [67]:
simple_eta_C_wnd([(0.5,0.7),(0.6,1.0)], False, Orientation(0.4, 0.3, 0.4, 0.3) )

Orientation(D0=0.59999999999999998, D90=0.59999999999999998, D180=0.59999999999999998, D270=0.59999999999999998)

## 温度差係数

#### 定義

In [68]:
def get_H(ASType, region):
    # ASType: 隣接空間の種類 [ AdjacentSpaceType 列挙体 ]
    # resion: 地域の区分 [ Region 列挙体 ]
    if region in { Region.Division1, Region.Division2, Region.Division3 }: # 地域の区分が1～3 の場合
        h = {
            AdjacentSpaceType.External : 1.0,
            AdjacentSpaceType.SemiExternal : 0.7,
            AdjacentSpaceType.Internal : 0.05
        }
        return h[ASType]
    else:  # 地域の区分が4～8 の場合
        h = {
            AdjacentSpaceType.External : 1.0,
            AdjacentSpaceType.SemiExternal : 0.7,
            AdjacentSpaceType.Internal : 0.15
        }
        return h[ASType]

#### 説明

In [69]:
get_H(AdjacentSpaceType.Internal, Region.Division5 )

0.15

In [70]:
get_H(AdjacentSpaceType.Internal, Region.Division3 )

0.05

## 方位係数

#### 定義

In [71]:
def get_nu_H(region, direction):
    # region: 地域の区分 [ Resion 列挙体 ]
    # direction: 方位 [ Direction 列挙体 ]
    r = region.value # 1～8 地域の区分の番号に対応
    d = direction.value # 0～7:南,南西,...,南東 8:上向き 9:下向き
    
    S      = [ 0.935, 0.856, 0.851, 0.815, 0.983, 0.936, 1.023, 'ND' ][r-1]
    SW     = [ 0.790, 0.753, 0.750, 0.723, 0.815, 0.763, 0.848, 'ND' ][r-1]
    W      = [ 0.535, 0.544, 0.542, 0.527, 0.538, 0.523, 0.548, 'ND' ][r-1]
    NW     = [ 0.325, 0.341, 0.351, 0.326, 0.297, 0.317, 0.284, 'ND' ][r-1]
    N      = [ 0.260, 0.263, 0.284, 0.256, 0.238, 0.261, 0.227, 'ND' ][r-1]
    NE     = [ 0.333, 0.341, 0.348, 0.330, 0.310, 0.325, 0.281, 'ND' ][r-1]
    E      = [ 0.564, 0.554, 0.540, 0.531, 0.568, 0.579, 0.543, 'ND' ][r-1]
    SE     = [ 0.823, 0.766, 0.751, 0.724, 0.846, 0.833, 0.843, 'ND' ][r-1]
    top    = [ 1.0,   1.0,   1.0,   1.0,   1.0,   1.0,   1.0,   'ND' ][r-1]
    bottom = [ 0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   'ND' ][r-1]
    
    return [S, SW, W, NW, N, NE, E, SE, top, bottom][d]

In [72]:
def get_nu_C(region, direction):
    # region: 地域の区分 [ Resion 列挙体 ]
    # direction: 方位 [ Direction 列挙体 ]
    r = region.value # 1～8 地域の区分の番号に対応
    d = direction.value # 0～7:南,南西,...,南東 8:上向き 9:下向き 

    S      = [ 0.502, 0.507, 0.476, 0.437, 0.472, 0.434, 0.412, 0.480 ][r-1]
    SW     = [ 0.526, 0.548, 0.550, 0.481, 0.520, 0.491, 0.479, 0.517 ][r-1]
    W      = [ 0.508, 0.529, 0.553, 0.481, 0.518, 0.504, 0.495, 0.505 ][r-1]
    NW     = [ 0.411, 0.428, 0.447, 0.401, 0.442, 0.427, 0.406, 0.411 ][r-1]
    N      = [ 0.329, 0.341, 0.335, 0.322, 0.373, 0.341, 0.307, 0.325 ][r-1]
    NE     = [ 0.430, 0.412, 0.390, 0.426, 0.437, 0.431, 0.415, 0.414 ][r-1]
    E      = [ 0.545, 0.503, 0.468, 0.518, 0.500, 0.512, 0.509, 0.515 ][r-1]
    SE     = [ 0.560, 0.527, 0.487, 0.508, 0.500, 0.498, 0.490, 0.528 ][r-1]
    top    = [ 1.0,   1.0,   1.0,   1.0,   1.0,   1.0,   1.0,   1.0   ][r-1]
    bottom = [ 0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0   ][r-1]
    
    return [S, SW, W, NW, N, NE, E, SE, top, bottom][d]

#### 説明

In [73]:
get_nu_H(Region.Division1, Direction.E)

0.564

In [74]:
get_nu_C(Region.Division1, Direction.E)

0.545

# 外皮性能の計算方法 (統合)

## 詳細法

In [75]:
def EnvPerformance(env_parts, env_HBs, env_EF_Peris, A_EF, A_A, region, ):

    ##### 面積関係の計算 #####
    
    # 外皮の表面積の合計(m2)
    a_env = get_A_env( [ env_part.A for env_part in env_parts ], A_EF )
    print( "外皮の面積の合計: " + str(a_env) )
    
    # 床面積の合計に対する外皮の部位の比
    r_env = get_r_env( a_env, A_A )
    print( "床面積の合計に対する外皮の部位の比: " + str(r_env) )
    
    ##### UA値、Q'値の計算 #####

    # ( 面積, U値, 温度差係数 ) の配列を作成
    # 温度差係数は、隣接空間の種類と地域の区分から決定
    U_parts = [ (env_part.A, env_part.U, get_H(env_part.ASType, region) ) for env_part in env_parts ]
    print( "U_parts: " + str(U_parts) )
    
    # ( 長さ, psi値, 温度差係数 ) の配列を作成
    # 温度差係数は、隣接空間の種類と地域の区分から決定
    psi_parts1 = [ (env_HB.L, env_HB.psi, get_H(env_HB.ASType, region) ) for env_HB in env_HBs ]
    print( "psi_parts1: " + str(psi_parts1) )
    
    # ( 長さ, psi値, 温度差係数 ) の配列を作成
    # 温度差係数は、隣接空間の種類と地域の区分から決定
    psi_parts2 = [ (env_EF_Peri.L, env_EF_Peri.psi, get_H(env_EF_Peri.ASType, region) ) for env_EF_Peri in env_EF_Peris ]
    print( "psi_parts2: " + str(psi_parts2) )

    # UA値(W/m2K)
    UA = get_UA( U_parts, psi_parts1 + psi_parts2, a_env )
    print( "UA値: " + str(UA) )
    
    # 熱損失係数(換気による熱損失を含まない) 
    Q_dash = get_Q_dash( float(UA), r_env )
    print( "熱損失係数(換気による熱損失を含まない) : " + str(Q_dash) )
    
    ##### ηA値、m値の計算 (暖房期) #####

    # ( 面積, 日射熱取得率, 方位係数 ) の配列を作成
    eta_H_parts = [ (env_part.A, env_part.eta_H, get_nu_H(region, env_part.direction) ) for env_part in env_parts ]
    print( "eta_H_parts: " + str(eta_H_parts) )

    # ( 面積, 熱橋の日射熱取得率, 方位係数1, 方位係数2 ) の配列を作成
    eta_H_dash_HBs = [ (env_HB.L, env_HB.eta_H_dash, get_nu_H(region, env_HB.direction1), get_nu_H(region, env_HB.direction2) ) for env_HB in env_HBs ]
    print( "eta_H_dash_HBs: " + str(eta_H_dash_HBs) )

    # ηAH値(-)
    eta_A_H = get_eta_A_H(eta_H_parts, eta_H_dash_HBs, a_env)
    print( "eta_A_H: " + str(eta_A_H) )
    
    # μH値(-)
    mu_H = getMuH(float(eta_A_H), r_env)
    print( "mu_H: " + str(mu_H) )
    
    ##### ηA値、m値の計算 (冷房期) #####

    # ( 面積, 日射熱取得率, 方位係数 ) の配列を作成
    eta_C_parts = [ (env_part.A, env_part.eta_C, get_nu_C(region, env_part.direction) ) for env_part in env_parts ]
    print( "eta_C_parts: " + str(eta_C_parts) )
    
    # ( 面積, 熱橋の日射熱取得率, 方位係数1, 方位係数2 ) の配列を作成
    eta_C_dash_HBs = [ (env_HB.L, env_HB.eta_C_dash, get_nu_C(region, env_HB.direction1), get_nu_C(region, env_HB.direction2) ) for env_HB in env_HBs ]
    print( "eta_C_dash_HBs: " + str(eta_C_dash_HBs) )    

    # ηAC値(-)
    eta_A_C = get_eta_A_C(eta_C_parts, eta_C_dash_HBs, a_env)
    print( "eta_A_C: " + str(eta_A_C) )

    # mC値(-)
    mu_C = getMuC(float(eta_A_C), r_env)
    print( "mu_C: " + str(mu_C) )


In [76]:
env_part_1 = EnvPart(A=1.0,U=1.0,eta_H=1.0,eta_C=1.0,direction=Direction.E,ASType=AdjacentSpaceType.External)
env_part_2 = EnvPart(A=1.0,U=2.0,eta_H=1.0,eta_C=1.0,direction=Direction.E,ASType=AdjacentSpaceType.External)
env_part_3 = EnvPart(A=1.0,U=3.0,eta_H=1.0,eta_C=1.0,direction=Direction.E,ASType=AdjacentSpaceType.External)
env_HB_1 = EnvHB(L=1.0,psi=1.0,eta_H_dash=1.0,eta_C_dash=1.0,direction1=Direction.E,direction2=Direction.S,ASType=AdjacentSpaceType.External)
env_HB_2 = EnvHB(L=1.0,psi=1.0,eta_H_dash=1.0,eta_C_dash=1.0,direction1=Direction.E,direction2=Direction.S,ASType=AdjacentSpaceType.External)
env_HB_3 = EnvHB(L=1.0,psi=1.0,eta_H_dash=1.0,eta_C_dash=1.0,direction1=Direction.E,direction2=Direction.S,ASType=AdjacentSpaceType.External)
env_EF_Peri_1 = EnvEFPeri(L=1.0,psi=1.0,ASType=AdjacentSpaceType.External)
env_EF_Peri_2 = EnvEFPeri(L=1.0,psi=1.0,ASType=AdjacentSpaceType.External)
env_EF_Peri_3 = EnvEFPeri(L=1.0,psi=1.0,ASType=AdjacentSpaceType.External)

EnvPerformance( [ env_part_1, env_part_2, env_part_3 ],
                [ env_HB_1, env_HB_2, env_HB_3 ],
                [ env_EF_Peri_1, env_EF_Peri_2, env_EF_Peri_3 ],
                [ 16.0 ], 30.0, Region.Division6 )

外皮の面積の合計: 19.0
床面積の合計に対する外皮の部位の比: 0.6333333333333333
U_parts: [(1.0, 1.0, 1.0), (1.0, 2.0, 1.0), (1.0, 3.0, 1.0)]
psi_parts1: [(1.0, 1.0, 1.0), (1.0, 1.0, 1.0), (1.0, 1.0, 1.0)]
psi_parts2: [(1.0, 1.0, 1.0), (1.0, 1.0, 1.0), (1.0, 1.0, 1.0)]
UA値: 0.64
熱損失係数(換気による熱損失を含まない) : 0.4053333333333333
eta_H_parts: [(1.0, 1.0, 0.579), (1.0, 1.0, 0.579), (1.0, 1.0, 0.579)]
eta_H_dash_HBs: [(1.0, 1.0, 0.579, 0.936), (1.0, 1.0, 0.579, 0.936), (1.0, 1.0, 0.579, 0.936)]
eta_A_H: 21.1
mu_H: 0.13363333333333333
eta_C_parts: [(1.0, 1.0, 0.512), (1.0, 1.0, 0.512), (1.0, 1.0, 0.512)]
eta_C_dash_HBs: [(1.0, 1.0, 0.512, 0.434), (1.0, 1.0, 0.512, 0.434), (1.0, 1.0, 0.512, 0.434)]
eta_A_C: 15.6
mu_C: 0.09879999999999999
